In [1]:
%run tools.ipynb

### BaseFeaturizer and BaseDescriptor 

In [14]:
# --- import necessary libraries
from xenonpy.descriptor.base import BaseDescriptor, BaseFeaturizer

# --- Featurizer ---
class RadialDistributionFunction_(BaseFeaturizer):

    @property
    def feature_labels(self):
        return [str(d) for d in self._interval[1:]]

    def __init__(self, n_bins=201, r_max=20.0, *, n_jobs=-1, on_errors='raise', return_type='any'):

        super().__init__(n_jobs=n_jobs, on_errors=on_errors, return_type=return_type)

        self.n_bins = n_bins
        self.r_max = r_max
        self.dr = r_max / (n_bins - 1)
        self._interval = np.arange(0.0, r_max + self.dr, self.dr)
        self.__authors__ = ['TsumiNa']

    def featurize(self, structure):

        # Get the distances between all atoms
        neighbors_lst = structure.get_all_neighbors(self.r_max)
        all_distances = np.concatenate(tuple(map(lambda x: [e[1] for e in x], neighbors_lst)))

        # Compute a histogram
        dist_hist, dist_bins = np.histogram(all_distances, bins=self._interval, density=False)

        # Normalize counts
        shell_vol = 4.0 / 3.0 * np.pi * (np.power(dist_bins[1:], 3) - np.power(dist_bins[:-1], 3))
        number_density = structure.num_sites / structure.volume
        return dist_hist / shell_vol / number_density

# --- Descriptor ---
class Compositions_(BaseDescriptor):

    def __init__(self, elements=None, *, n_jobs=-1, include=None,
                 exclude=None, on_errors='raise'):
        super().__init__()
        self.n_jobs = n_jobs

        self.composition = WeightedAvgFeature(elements, n_jobs=n_jobs, on_errors=on_errors)
        self.composition = WeightedSumFeature(elements, n_jobs=n_jobs, on_errors=on_errors)
        self.composition = WeightedVarFeature(elements, n_jobs=n_jobs, on_errors=on_errors)
        self.composition = MaxFeature(elements, n_jobs=n_jobs, on_errors=on_errors)
        self.composition = MinFeature(elements, n_jobs=n_jobs, on_errors=on_errors)

### build your own

In [2]:
# --- import necessary libraries

from xenonpy.descriptor.base import BaseDescriptor, BaseFeaturizer

In [3]:
class YourFeaturier1(BaseFeaturizer):

    def __init__(self, n_jobs=1):
        super().__init__(n_jobs=n_jobs)

    def featurize(self, x):
        return x * 10

    @property
    def feature_labels(self):
        return ['times']

class YourFeaturier2(BaseFeaturizer):

    def __init__(self, n_jobs=1):
        super().__init__(n_jobs=n_jobs)

    def featurize(self, x):
        return x ** 2

    @property
    def feature_labels(self):
        return ['power']    

class YourDescriptor(BaseDescriptor):

    def __init__(self):
        super().__init__()
        self.f1 = YourFeaturier1()
        self.f2 = YourFeaturier2()

In [4]:
test_data1 = np.array([1, 2, 3, 4, 5])
test_data2 = np.array([6, 7, 8, 9, 10])

In [5]:
yf1 = YourFeaturier1()
yf1.fit_transform(test_data1, return_type='df')

,times
0,10
1,20
2,30
3,40
4,50


In [6]:
yf2 = YourFeaturier2()
yf2.fit_transform(test_data2, return_type='df')

,power
0,36
1,49
2,64
3,81
4,100


In [7]:
test_data = pd.DataFrame({'f1': test_data1, 'f2': test_data2})
test_data

,f1,f2
0,1,6
1,2,7
2,3,8
3,4,9
4,5,10


In [8]:
yd = YourDescriptor()
yd.fit_transform(test_data)

,times,power
0,10,36
1,20,49
2,30,64
3,40,81
4,50,100
